In [27]:
pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 7.5 MB/s  0:00:02 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [10]:
cd  //home/rajah/Documents/Yandex/Netology/Egorov_RV_Python_HomeWork

/home/rajah/Documents/Yandex/Netology/Egorov_RV_Python_HomeWork


In [11]:
ls


 HR.csv              HW6.html                    HW_visualisation.zip
'hw1 Python.html'    HW6.ipynb                   Kobe/
'hw1 Python.ipynb'   HW_alg_class.ipynb          Linear_models/
 HW2.html            HW_Data_quality.html        pairplot.png
 HW2.ipynb           HW_Data_quality.ipynb       Project_Python_Preprocessing/
 HW3.html            HW_Linear_regress_params/   Python_project.html
 HW3.ipynb           HW_optimization.html        Python_project.ipynb
 HW4.html            HW_optimization.ipynb       Untitled1.ipynb
 HW4.ipynb           HW_trees/                   Untitled.ipynb
 HW5.html            HW_visualisation.html      'Visualisation Data.ipynb'
 HW5.ipynb           HW_visualisation.ipynb


In [12]:
df_pd = pd.read_csv(
    'HR.csv', 
    engine='pyarrow', 
    dtype_backend='pyarrow'
)

In [13]:
print(df_pd['sales'].dtype) 

string[pyarrow]


In [14]:
res = df_pd.groupby('salary')['satisfaction_level'].mean()

In [15]:
res

salary
high       0.63747
low       0.600753
medium    0.621817
Name: satisfaction_level, dtype: double[pyarrow]

In [18]:
import polars as pl

In [20]:
df_pl = pl.read_csv('HR.csv')
result_eager = df_pl.filter(pl.col("left") == 1).group_by("sales").len()

In [23]:
df_pl.head()
result_eager

sales,len
str,u32
"""hr""",215
"""marketing""",203
"""technical""",697
"""IT""",273
"""sales""",1014
"""support""",555
"""RandD""",121
"""accounting""",204
"""management""",91


In [24]:
query = (
    pl.scan_csv('HR.csv')
    .filter(pl.col("satisfaction_level") > 0.5)
    .group_by("salary")
    .agg(pl.col("left").mean().alias("churn_rate"))
)

In [25]:
result_lazy = query.collect()
print(result_lazy)

shape: (3, 2)
┌────────┬────────────┐
│ salary ┆ churn_rate │
│ ---    ┆ ---        │
│ str    ┆ f64        │
╞════════╪════════════╡
│ medium ┆ 0.085252   │
│ low    ┆ 0.129941   │
│ high   ┆ 0.019672   │
└────────┴────────────┘


In [28]:
import duckdb

In [29]:
query = """
SELECT 
    salary, 
    AVG(satisfaction_level) as avg_sat,
    COUNT(*) as employee_count
FROM 'HR.csv'
GROUP BY salary
ORDER BY avg_sat DESC
"""

In [39]:
result = duckdb.query(query2).to_df()
result.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [35]:
query2 = """
SELECT * FROM 'HR.csv' LIMIT 10
"""

In [38]:
con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL xml; LOAD xml;")


HTTPException: HTTP Error: Failed to download extension "xml" at URL "http://extensions.duckdb.org/v1.4.3/linux_amd64/xml.duckdb_extension.gz" (HTTP 404)

Candidate extensions: "jemalloc", "mysql_scanner", "delta", "excel", "mysql"
For more info, visit https://duckdb.org/docs/stable/extensions/troubleshooting?version=v1.4.3&platform=linux_amd64&extension=xml